In [4]:
import pandas as pd
import geopandas as gpd

park = 'Aiguestortes'

# Path to the shapefile
shapefile_path = f'C:/Users/carlo/Documents/EarthCul/OSM/2) Derived metrics/count/results/{park}/grid_3035_count.shp'

# Read the shapefile using geopandas
df = gpd.read_file(shapefile_path)

# Suppose you have a DataFrame named df with 34 columns
# You can adjust this according to your DataFrame structure

# Get the list of column names
column_names = df.columns

# columns_to_process = df.columns.tolist()[10:] # Uncomment this line if you want to select only some columns

# Iterate over the columns and create new columns
for column_name in column_names: # Replace column_to_process here if you want to select only some columns
    # Create the name of the new column
    new_column_name = 'pre_' + column_name
    
    # Create the new column with values 0 or 1
    df[new_column_name] = df[column_name].apply(lambda x: 0 if x == 0 else 1)

# Now, df contains the new columns with the desired values

df.to_file(f'C:/Users/carlo/Documents/EarthCul/OSM/2) Derived metrics/presence/results/{park}/grid_3035_presence.shp', driver='ESRI Shapefile')


C:\Users\carlo\AppData\Local\Temp\ipykernel_14180\2721251281.py:30: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  df.to_file(f'C:/Users/carlo/Documents/EarthCul/OSM/2) Derived metrics/presence/results/{park}/grid_3035_presence.shp', driver='ESRI Shapefile')


In [5]:
from tqdm import tqdm
import os
import geopandas as gpd
import rasterio
from rasterio import features
Base_raster= f"C:/Users/carlo/Documents/EarthCul/OSM/2) Derived metrics/BaseLayers/{park}/Slope.tif" 
              
# Output folder
output_folder = f"C:/Users/carlo/Documents/EarthCul/OSM/2) Derived metrics/presence/results/{park}"  

gdf = gpd.read_file(f'C:/Users/carlo/Documents/EarthCul/OSM/2) Derived metrics/presence/results/{park}/grid_3035_presence.shp')

with rasterio.open(Base_raster) as src:
    profile = src.profile  

    # Loop through columns to rasterize 
    columns_to_process = gdf.columns.tolist()[34:]#select the correct columns
    for column in tqdm(columns_to_process, desc='Rasterizing Columns'):
        values = gdf[column].values
        
        # Rasterize the values
        rasterized = features.rasterize(
            zip(gdf.geometry, values),
            out_shape=(profile['height'], profile['width']),
            transform=profile['transform'],
            fill=-999.99,  
            all_touched=True,  
            default_value=0,  
            dtype=profile['dtype']  
        )
 
        output_path = os.path.join(output_folder, f'{column}.tif')
        
        # Save the outputs
        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(rasterized, 1)

Rasterizing Columns:  97%|█████████▋| 35/36 [11:55<00:20, 20.43s/it]


TypeError: '<=' not supported between instances of 'Polygon' and 'Polygon'